In [1]:
from caveclient import CAVEclient
from intern import array
import pickle
import numpy as np
from agents import data_loader
from cloudvolume import CloudVolume
from membrane_detection import membranes
from agents.scripts import precompute_membrane_vectors, create_post_matrix, merge_paths, get_soma
import agents.sensor
from agents.run import run_agents
import aws.sqs as sqs
import sys
import time
import ast
import pandas as pd
import agents.scripts as scripts
from drive import drive
from finding_orphans import *
from math import sqrt
from tip_finding.tip_finding import tip_finder_decimation

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load = pd.read_csv("agents/Data/endpoints_gt2.csv")
endpoints_dict = {}

In [3]:
load.head()

,neuron,ng_link,seg_id,pink_pts,num_endpoints,Endpoints,Comments
0,8.646911e+17,https://neuroglancer.neuvue.io/?json_url=https...,864691135909994000,"(402188, 228684, 24029)",2,"((402584, 228856, 23991), (402985, 229235, 235...",NaN
1,NaN,NaN,864691135247440303,"(401258, 224832, 24029)",3,"((401612, 224623, 23991), (405257, 226318, 236...",NaN
2,NaN,NaN,864691134794123793,"(401314, 228366, 24424)",2,"((401242, 228382, 24444), (400982, 228457, 245...",NaN
3,NaN,NaN,864691134959738482,"(400199, 220721, 24029)",7,"((401289, 218721, 23991), (399895, 216533, 235...",NaN
4,NaN,NaN,864691135314714227,"(397870, 232292, 24004)",2,"((397867, 232230, 23999), (397854, 232252, 239...",NaN


In [5]:
#creating series objects for the desired collumns 
segIDs = load['seg_id']
Endpoints = load['Endpoints']


In [ ]:
for idx, i in enumerate(load['seg_id']):
    try:
        t1, skel, mesh_obj = tip_finder_decimation(str(i))
        endpoints_dict[i] = t1
    except:
        print(f"\n\nSeg {i} returned error on get. Skipping.\n")
        pass

load['Endpoints'] = load['Endpoints'].apply(lambda x: list(ast.literal_eval(x)))
load['proposed_endpoints'] = load.seg_id.map(endpoints_dict)

In [ ]:
#correctly  have 2 collumns for the proposed vs gt endpoints
load.head()

In [8]:

def find_euclidian_distance(proposed_endpoint, gt_endponts_array):
    smallest_distance = 2**30
    for endpoint in gt_endponts_array:
        difference = proposed_endpoint - endpoint
        distance = np.sqrt(np.sum(np.square(difference)))
        if(distance < smallest_distance):
            smallest_distance = distance 
    return smallest_distance 
            
    
     


In [36]:
#TEST 1: invalidation_d at default: 12000, WITH Humfrey_Smoothing: 50 iterations, Decimation_Factor = 0.50
for index, row in load.iterrows():
    threshold = 200
    endpoint_array = np.array(row["Endpoints"])
    proposed_endpoints_array = np.array(row["proposed_endpoints"])
    print(endpoint_array)
    print()
    print(proposed_endpoints_array)
    print()
    #print(load["Endpoints"])
    #check if there are no proposed endpoints

    if (len(proposed_endpoints_array.shape) == 0 and len(endpoint_array.shape) == 0):
        print("correct")
    elif (len(proposed_endpoints_array.shape) == 0 and len(endpoint_array) > 0):
        print("not correct")
        continue 
    
    # if len(proposed_endpoints_array.shape)and not endpoint_array:
    #    print("correct")
    # elif not proposed_endpoints_array and endpoint_array.size > 0:
    #     print("wrong")
    

    

    for proposed_endpoint in proposed_endpoints_array:
        smallest_distance= find_euclidian_distance(proposed_endpoint, endpoint_array)
        print(smallest_distance)
        
        
        #compare to threshold and return yes or no 
        if smallest_distance < threshold:
            print("correct")
            print()
        else:
            print("wrong")
            print()
    

[[402584 228856  23991]
 [402985 229235  23554]]

[[402567 228758  23991]
 [402990 229222  23552]]

99.46356116689167
correct

14.071247279470288
correct

[[401612 224623  23991]
 [405257 226318  23620]
 [409162 219642  23891]]

[[401467 224644  23990]
 [405021 226754  23620]
 [409143 219355  23868]]

146.5162107072115
correct

495.77414212522217
wrong

288.54635676091976
wrong

[[401242 228382  24444]
 [400982 228457  24513]]

[[400269 228563  24514]
 [400319 228912  24514]]

720.8370134780816
wrong

804.1113107026912
wrong

[[401289 218721  23991]
 [399895 216533  23526]
 [406366 215719  23621]
 [403940 210639  23991]
 [394569 213255  23991]
 [376682 175983  22020]
 [365111 191189  24020]]

[[400448 220975  24009]
 [401240 217916  24008]]

2405.851408545424
wrong

806.6690771313848
wrong

[[397867 232230  23999]
 [397854 232252  23997]]

[[397815 232393  23996]
 [397931 232013  24000]]

146.29764181284673
correct

226.24323194296886
wrong

[[403332 227545  24398]
 [403566 227614  244

make threshold for each endpoint
experiment with box (cannot overlap)
for each endpoint see if it finds in box -- if yes true found endpont, if false --not an endpoint
find distance from predicted to ground truth 
could still do MSE 

In [ ]:
coords_diff = coords_array - coord_gt
coords_amin = np.argmin(np.sqrt(np.sum(np.square(coords_diff), axis=1)))



Threshold: 
-find the size of the segment (maybe connected componenets)
-array from the skel
-flat is edge case 
-maybe a percentage 
--a hard number 





In [33]:
if len(proposed_endpoints_array.shape) == 0:
    print("correct")


correct


In [ ]:
#what situation i succedd and fail 
#some kind of orphan it works/doesn't wor ofor 
#could be more than just dendrite 